##### Quantum Genetic Algorithm Proof of Concept

This is an experiment to initialize a chromosome using qubits, the hadamard gate and a QasmSimulator provided by the qiskit library.

In [44]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import QasmSimulator


num_of_design_variables = 1
num_of_qubits_per_design_variable = 12
num_of_qubits = num_of_design_variables * num_of_qubits_per_design_variable

q = QuantumRegister(num_of_qubits)
c = ClassicalRegister(num_of_qubits)
qc = QuantumCircuit(q,c)

for i in range(num_of_qubits):
  qc.h(i)

for i in range(num_of_qubits):
    qc.measure(q[i], c[i])

backend = QasmSimulator()
qc_compiled = transpile(qc, backend)
job = backend.run(qc_compiled, shots=1)
result = job.result()
counts = result.get_counts()

qubit_string = list(counts.keys())[0]
print('Sampled Bit String:', qubit_string)


Sampled Bit String: 000010101011


##### Storage caclulation for quantum register of variable number of qubits

We wanted to model n number of design variables with m number of bits each. In this case we would need n\*m qubits. Creating a single quantum register with n\*m qubits wouldn't work.

In order to store n\*m qubits we would need to store 2^(n\*m + 1) real numbers (4 bytes each). So for 4 design variables modelled by a 12bit chromosome each we would need 2Pb of storage, which is prohibitive and unecessary.

This means that we need to handle the chromosome of each design variable separately.

In [45]:
def convert_storage(storage):
  units = ['bytes', 'KB', 'MB', 'GB', 'TB', 'PB', 'EB', 'ZB', 'YB', 'BB', 'NB', 'DB', 'CB']
  unit_index = 0
  while storage >= 1024 and unit_index < len(units) - 1:
    storage /= 1024
    unit_index += 1
  return storage, units[unit_index]

n = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # number of design variables
m = 12                              # number of qubits per design variable

for n_i in n:
  num_of_qubits = n_i * m
  num_of_reals = 2**(num_of_qubits + 1)
  storage_needed = num_of_reals * 4
  storage, unit = convert_storage(storage_needed)
  print(f'Storage for {n_i} design variables: {storage:.2f} {unit}')


TypeError: can only concatenate list (not "int") to list